In [ ]:
import pandas as pd
import numpy as np
#import mssev as ms

In [ ]:
data = pd.read_csv("Complete_Clinical_Data_12000.csv")
data.info()

In [ ]:
# Make the MSSS feature
data["MSSS"] = ms.MSSS(data, 
                       ds="EDSS_score_assessed_by_clinician", 
                       duration="num_year_from_onset")
data.dropna(subset=['MSSS'], axis = 0, inplace = True)

bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

data["MSSS_classified"] = pd.cut(data['MSSS'], 
                                 bins=bins, 
                                 labels=labels, 
                                 include_lowest=True, 
                                 right=True)
data["MSSS_classified"].hist()

In [ ]:
#Make outcome feature
def classify_outcome(x):
    if pd.isnull(x):
        return 'First obs'
    elif x == 0.5:
        return 'Very Mild Progression'
    elif x == 1:
        return 'Mild progression'
    elif x == 1.5:
        return 'Moderate progression'
    elif x < 0:
        return 'Improvment'
    elif x == 0:
        return 'No change'
    else:
        return 'Severe Progression'
    
data['num_obs'] = data.groupby('Patient_ID')['Patient_ID'].transform('count')
data = data[dat
            a.num_obs > 1]
data['Diff_EDSS'] = data.groupby('Patient_ID')['EDSS'].diff()
data[['Patient_ID','Date_of_visit','EDSS','Diff_EDSS']]
data['Outcome'] = data['Diff_EDSS'].map(classify_outcome)
data['Outcome'] = data['Outcome'].map({'First obs':0,
                                       'Improvment':1,
                                       'Very Mild Progression':1,
                                       'Mild progression':2,
                                       'Moderate progression':3,
                                       'No change':4,
                                       'Severe Progression':5})
data.head()

In [ ]:
#Add onset date,birth of day and CAP features
first_last_data = pd.read_csv("First_Last_Observation_Patient.csv")
data = pd.merge(data,
                first_last_data, 
                left_on="Patient_ID", 
                right_on="Patient_ID", 
                how = "left")

In [ ]:
#Making time feature
data['Date_of_visit'] = pd.to_datetime(data['Date_of_visit'],format ='%Y-%m-%d')
data["Date_of_Onset"] = pd.to_datetime(data["Date_of_Onset"],format ='%Y-%m-%d')
data['year_from_onset'] = ((data['Date_of_visit'] - data['Date_of_Onset']) / np.timedelta64(1, 'Y')).round(2)
data.head()

In [ ]:
#Making Patient number feature
data['Patient_number'] = data.groupby('Patient_ID').ngroup() + 1

In [ ]:
#Save the final clinical data
data.to_csv("Completed_data_with_Progression_Outcome.csv", index = False)